# Agent Based Model: Main Notebook

In [125]:
import numpy.random as rnd
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Creating Study Groups

#### Practical Functions

In [126]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(2.5, 2.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000)

In [136]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    ID_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = [] 
    social_list = []
    
    for student in studygroup:
        ID_list.append(student.ID)
        extraversion_list.append(student.Extraversion())
        sensing_list.append(student.Sensing()) 
        thinking_list.append(student.Thinking())
        judging_list.append(student.Judging())
    
    
    for i in range(len(studygroup)):
        academic_list.append(student.Academic_Skill(extraversion_list[i], sensing_list[i], thinking_list[i], judging_list[i]))
        social_list.append(student.Social_Skill(extraversion_list[i], sensing_list[i], thinking_list[i], judging_list[i]))

    data = pd.DataFrame({'ID': ID_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list,
                        'Social': social_list})
    
    return data
    

#### Create Study Group

In [137]:
## Defining the Students ##
class Student():
    def __init__(self, ID, Ex, Se, Th, Ju):
        self.ID  = ID
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension

    def Extraversion(self):
        if self.Ex == "E":
            self.ExScore = max_vals[0]
            del max_vals[0]
            return self.ExScore
        else:
            self.ExScore = min_vals[0]
            del min_vals[0]
            return self.ExScore

    def Sensing(self):
        if self.Se == "S":
            self.SeScore = max_vals[0]
            del max_vals[0]
            return self.SeScore
        else:
            self.SeScore = min_vals[0]
            del min_vals[0]
            return self.SeScore
    
    def Thinking(self):
        if self.Th == "T":
            self.ThScore = max_vals[0]
            del max_vals[0]
            return self.ThScore
        else: 
            self.ThScore = min_vals[0]
            del min_vals[0]
            return self.ThScore
    
    def Judging(self):
        if self.Ju == "J":
            self.JuScore = max_vals[0]
            del max_vals[0]
            return self.JuScore
        else: 
            self.ThScore = min_vals[0]
            del min_vals[0]
            return self.ThScore
    
    def Academic_Skill(self, ExScore, SeScore, ThScore, JuScore):
        return (1-ExScore)*0.15 + (1-SeScore)*0.35 + ThScore*0.20 + (1-JuScore)*0.30
        
    def Social_Skill(self, ExScore, SeScore, ThScore, JuScore):
        return ExScore*0.50 + (1-SeScore)*0.10 + ThScore*0.30 + (1-JuScore)*0.10

In [138]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []

    for i in range(len(student_list)):
        student = Student(i+1, student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        studygroup.append(student)
    
    return data_collect(studygroup, student_list)

In [139]:
student_list_test = ["ESTJ", "ISTJ", "ENTP", "INFJ"]

studygroup1 = StudyGroup(student_list_test)


print(studygroup1)

   ID  type       E/I       S/N       T/F       J/P  Academic    Social
0   1  ESTJ  0.875363  0.740621  0.607738  0.700933  0.320746  0.675847
1   2  ISTJ  0.225654  0.618409  0.893140  0.669063  0.527618  0.452022
2   3  ENTP  0.902976  0.393821  0.711792  0.307600  0.576795  0.794883
3   4  INFJ  0.289985  0.248742  0.447722  0.537221  0.597821  0.400713


In [140]:
student_list_test_2 = ["ESTJ", "ISTJ", "ENTP", "INFJ"]

studygroup2 = StudyGroup(student_list_test_2)


print(studygroup2)

   ID  type       E/I       S/N       T/F       J/P  Academic    Social
0   1  ESTJ  0.546684  0.880651  0.533365  0.632683  0.326638  0.482018
1   2  ISTJ  0.372075  0.861868  0.729482  0.580414  0.414307  0.460654
2   3  ENTP  0.609255  0.379271  0.521725  0.095022  0.651705  0.613716
3   4  INFJ  0.248474  0.023402  0.373501  0.865302  0.569648  0.347417


In [141]:
student_list_test_3 = ["ESTJ", "ENTJ", "ENTP", "ESTP"]

studygroup3 = StudyGroup(student_list_test_3)

print(studygroup3)

   ID  type       E/I       S/N       T/F       J/P  Academic    Social
0   1  ESTJ  0.704706  0.506880  0.609363  0.544241  0.475487  0.630050
1   2  ENTJ  0.754587  0.393224  0.980034  0.644578  0.551817  0.767524
2   3  ENTP  0.682566  0.382862  0.710270  0.309196  0.612909  0.685158
3   4  ESTP  0.565736  0.752383  0.781182  0.468578  0.467469  0.595126


## Creating the ABM

In [103]:
def true_solution(n_elements, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    solution = []
    for i in range(n_elements):
        solution.append(random.randint(range_elements[0], range_elements[1]))

    return solution

In [143]:
true_solution(8, [1,9])
studygroup3["Academic"]

0    0.475487
1    0.551817
2    0.612909
3    0.467469
Name: Academic, dtype: float64

In [146]:
def individual_solutions(student_df, true_solution):
    '''
    function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''

    academic_skills = student_df["Academic"]

    for i in range(len(academic_skills)):
        indi_solution = []
        academic_skill = academic_skills[i]
        
        for i in range(len(true_solution)):
            pass